<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Compressive-Transformer" data-toc-modified-id="Compressive-Transformer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Compressive Transformer</a></span><ul class="toc-item"><li><span><a href="#Options:" data-toc-modified-id="Options:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Options:</a></span></li><li><span><a href="#setup" data-toc-modified-id="setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#Create-Model" data-toc-modified-id="Create-Model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create Model</a></span></li><li><span><a href="#Model-summary" data-toc-modified-id="Model-summary-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Model summary</a></span></li><li><span><a href="#Create-train/test-data-generators" data-toc-modified-id="Create-train/test-data-generators-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Create train/test data-generators</a></span></li><li><span><a href="#Create-addtional-Run-Configurations" data-toc-modified-id="Create-addtional-Run-Configurations-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Create addtional Run Configurations</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Train</a></span></li></ul></li></ul></div>

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep, time
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.models import load_model
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda, Add
from keras.callbacks import Callback
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep, time
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.models import load_model
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda, Add
from keras.callbacks import Callback
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from tqdm.notebook import tqdm
from sometimer import timer, time_this_method

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention
from model.layers import content_based_attention

from model import CompressiveTransformer
from model import AttentionReconstruction

from model.callbacks import ClearCompressedMemory, WriteLogsToFile
from train.generators import next_token_batch_generator
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from tqdm.notebook import tqdm
from sometimer import timer, time_this_method

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
import preprocess
# from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention
from model.layers import content_based_attention

from model import CompressiveTransformer
from model import AttentionReconstruction

from model.callbacks import ClearCompressedMemory, WriteLogsToFile
from train.generators import next_token_batch_generator

Using TensorFlow backend.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compressive Transformer

## Options:

In [2]:
# Preprocessing
vocab_size=16384
lowercase=False
language='english'
train_val_split=0.8
input_paths={'en': '../data/wma-en-de/input/train-en.txt',
             'de': '../data/wma-en-de/input/train-de.txt'}
tokenizer_output_path=f'../data/wma-en-de/tokenizer/' \
                      f'en-de-v0-t{vocab_size}{"-lowercase" if lowercase else ""}.tok'

In [3]:
epochs=100
batch_size=128
d_layers=2
sequence_length=128
d_model=128
train_steps=1200000
validation_steps=10000

## setup

In [4]:
wma = load.wma(input_paths['en'], 
               input_paths['de'])

In [5]:
tokenizer = Tokenizer(input_paths=list(input_paths.values()),
                      tokenizer_output_path=tokenizer_output_path,
                      vocab_size=vocab_size,
                      lowercase=lowercase)

In [6]:
_, _, train, val = preprocess.wma(input_paths,
                                  wma[[language]],
                                  tokenizer_output_path,
                                  train_val_split=train_val_split,
                                  vocab_size=vocab_size,
                                  language=language,
                                  lowercase=lowercase,
                                  tokenizer=tokenizer)

In [7]:
epoch_steps=train_steps or len(train)
validation_steps=validation_steps or len(val)
steps_per_epoch=epoch_steps//sequence_length - 1
steps_per_validation=validation_steps//sequence_length - 1

## Create Model

In [8]:
ct = CompressiveTransformer(d_layers=d_layers,
                            sequence_length=sequence_length, 
                            d_model=d_model,
                            memory_size=256,
                            compressed_memory_size=256,
                            d_k=16, 
                            d_heads=2, 
                            output_size=vocab_size,
                            batch_size=batch_size,
                            vocab_size=vocab_size)

In [9]:
ct.compile(optimizer='Adam',
           loss='categorical_crossentropy',
           metrics=['accuracy']
          )

## Model summary

In [10]:
ct.summary()

Model: "CompressiveTransformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 128)          0                                            
__________________________________________________________________________________________________
h_L0 (Embedding)                (None, 128, 128)     2097152     x[0][0]                          
__________________________________________________________________________________________________
compressed_memory (InputLayer)  (None, 2, 256, 128)  0                                            
__________________________________________________________________________________________________
memory (InputLayer)             (None, 2, 256, 128)  0                                            
_____________________________________________________________________________

## Create train/test data-generators

In [11]:
generator = next_token_batch_generator(ct=ct,
                                       data=train,
                                       data_path=None,
                                       epoch_steps=epoch_steps, 
                                       sequence_length=sequence_length, 
                                       batch_size=sequence_length,
                                       vocab_size=vocab_size)
validation_generator = next_token_batch_generator(ct=ct, 
                                                  data=val,
                                                  data_path=None,
                                                  epoch_steps=validation_steps, 
                                                  sequence_length=sequence_length, 
                                                  batch_size=sequence_length,
                                                  vocab_size=vocab_size)

## Create addtional Run Configurations

In [12]:
callbacks = [ClearCompressedMemory(),
             WriteLogsToFile(filepath='training-logs/ct.txt', overwrite_old_file=False)]

## Train

In [ ]:
ct.fit_generator(generator(), 
                 steps_per_epoch=steps_per_epoch,
                 epochs=epochs,
                 callbacks=callbacks,
                 validation_data=validation_generator(),
                 validation_steps=steps_per_validation,
                 shuffle=False
                 )

/home/vs/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
5036/9374 [===============>..............] - ETA: 29:19 - loss: 7.2159 - accuracy: 0.0367

- multiple layers
- larger embedding
- longer dataset
- relative embedding
- attention reconstruction loss